# 1、字符串输出解析器 StrOutputParser

In [2]:
# 1、获取大模型
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser, XMLOutputParser

import os
import dotenv
from langchain_core.utils import pre_init
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini")

# 2、调用大模型
response = chat_model.invoke("什么是大语言模型？")
# print(type(response))   #AIMessage

#3、如何获取一个字符串的输出结果呢？
# 方式1：自己调用输出结果的content
# print(response.content)

# 方式2：使用StrOutputParser
parser = StrOutputParser()
str_response = parser.invoke(response)
print(type(str_response))  #<class 'str'>
print(str_response)


<class 'str'>
大语言模型（Large Language Model, LLM）是一种基于深度学习技术的人工智能模型，旨在理解和生成自然语言文本。这类模型通常使用海量的文本数据进行训练，学习语言的结构、语法和语义，以便能够生成连贯、符合上下文的文本。

大语言模型的特点包括：

1. **规模庞大**：通常包含亿万或甚至数百亿个参数，这使得模型能够捕捉复杂的语言模式。

2. **预训练与微调**：模型首先在大规模的文本数据集上进行无监督的预训练，然后可以根据具体任务（如文本分类、问答生成等）进行微调。

3. **多样化应用**：可用于文本生成、翻译、问答、对话系统、内容推荐等多种自然语言处理任务。

4. **上下文理解能力**：能够理解上下文信息，生成相关的和一致的回应，使得与人类的互动更加自然。

大语言模型的代表有OpenAI的GPT系列、Google的BERT和T5以及Meta的LLaMA等。尽管这些模型表现出色，但也存在一些挑战，如易产生偏见、缺乏常识推理能力和生成不准确或不合适的内容等。


# 2、JsonOutputParser : Json输出解析器

方式1：

In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="gpt-4o-mini")

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 正确的：
prompt = chat_prompt_template.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})

# 错误的：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？"})

response = chat_model.invoke(prompt)
# print(response.content)

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

json_result = parser.invoke(response)
print(json_result)

```json
{
  "q": "人工智能用英文怎么说？",
  "a": "Artificial Intelligence"
}
```
{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}


方式2：

举例1：

In [9]:
parser = JsonOutputParser()

print(parser.get_format_instructions())

Return a JSON object.


举例2：

In [11]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

#以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = prompt_template.invoke(input={"question": joke_query})
response = chat_model.invoke(prompt)
print(response)

json_result = parser.invoke(response)
print(json_result)

content='```json\n{\n  "joke": "为什么鸡过马路？因为它想去对面！"\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 32, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-C94MgvoSBRdCxkG0J6lBy3t0QJAoc', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--85cc489c-68f5-4e26-bcda-ec0fa27a683e-0' usage_metadata={'input_tokens': 32, 'output_tokens': 27, 'total_tokens': 59, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
{'joke': '为什么鸡过马路？因为它想去对面！'}


知识的拓展： |

In [12]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="gpt-4o-mini")

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

# 写法1：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})
#
# response = chat_model.invoke(prompt)
#
# json_result = parser.invoke(response)
# print(json_result)


# 写法2：
chain = chat_prompt_template | chat_model | parser
json_result1 = chain.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})
print(json_result1)

{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}


针对于举例2

In [13]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

#以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
# 写法1：
# prompt = prompt_template.invoke(input={"question":joke_query})
# response = chat_model.invoke(prompt)
# json_result = parser.invoke(response)

chain = prompt_template | chat_model | parser
json_result = chain.invoke(input={"question": joke_query})
print(json_result)

{'joke': '为什么数学书总是感到沮丧？因为它有太多的问题！'}


# 3、XMLOutputParser XML输出解析器的使用

举例1：自己在提示词模板中写明使用XML格式

In [15]:
chat_model = ChatOpenAI(model="gpt-4o-mini")

actor_query = "周星驰的简短电影记录"
response = chat_model.invoke(f"请生成{actor_query}，将影片附在<movie></movie>标签中")

print(type(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
周星驰（Stephen Chow）是一位著名的中国演员、导演和编剧，以其独特的喜剧风格和幽默才华而闻名。以下是他的几部经典电影记录：

<movie>
  <title>大话西游之月光宝盒</title>
  <year>1995</year>
  <description>这是一部融合了喜剧、爱情和奇幻元素的经典电影，讲述了至尊宝和紫霞仙子的爱情故事，打破了传统的西游记情节，成为华语电影的经典之作。</description>
</movie>

<movie>
  <title>功夫</title>
  <year>2004</year>
  <description>这部电影是一部结合了功夫与喜剧的影片，讲述了一名小混混在意外中走上武道之路，最终成长为一名英雄的故事。该片因其独特的视听效果和幽默风格受到广泛好评。</description>
</movie>

<movie>
  <title>食神</title>
  <year>1996</year>
  <description>影片讲述了一位失去厨艺的明星厨师在艰难的环境中重拾信心与技艺，并最终成为食神的励志故事，融合了美食和搞笑元素。</description>
</movie>

<movie>
  <title>少林足球</title>
  <year>2001</year>
  <description>这是一部将足球与少林功夫结合的喜剧电影，讲述了一群绝望的少林寺弟子通过足球重拾信心的故事，展现了团队的力量和幽默。</description>
</movie>

<movie>
  <title>喜剧之王</title>
  <year>1999</year>
  <description>本片是一部关于追求梦想的喜剧，讲述了一个在电影界挣扎奋斗的年轻人，最终找到自我价值和爱情的感人故事。</description>
</movie>
</movie>


举例2：

In [17]:
from langchain_core.output_parsers.xml import XMLOutputParser

parser = XMLOutputParser()
print(parser.get_format_instructions())

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


使用parser.get_format_instructions()结构实现：

In [19]:
# 1.导入相关包
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 2. 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 3.测试模型的xml解析效果
actor_query = "生成汤姆·汉克斯的简短电影记录,使用中文回复"

# 4.定义XMLOutputParser对象
parser = XMLOutputParser()

# 5. 生成提示词模板
prompt_template1 = PromptTemplate.from_template(
    template="用户的问题：{query}\n使用的格式：{format_instructions}"
)

prompt_template2 = prompt_template1.partial(format_instructions=parser.get_format_instructions())


response = chat_model.invoke(prompt_template2.invoke(input={"query": actor_query}))
print(response.content)


```xml
<电影记录>
    <演员>
        <名字>汤姆·汉克斯</名字>
        <出生年份>1956</出生年份>
        <代表作品>
            <作品>《阿甘正传》</作品>
            <作品>《拯救大兵瑞恩》</作品>
            <作品>《失而复得》</作品>
            <作品>《外星人E.T.》</作品>
            <作品>《费城故事》</作品>
        </代表作品>
        <获奖经历>
            <奖项>奥斯卡金像奖</奖项>
            <奖项>金球奖</奖项>
            <奖项>BAFTA奖</奖项>
        </获奖经历>
    </演员>
</电影记录>
```


In [20]:
xml_result = parser.invoke(response)
print(xml_result)

{'电影记录': [{'演员': [{'名字': '汤姆·汉克斯'}, {'出生年份': '1956'}, {'代表作品': [{'作品': '《阿甘正传》'}, {'作品': '《拯救大兵瑞恩》'}, {'作品': '《失而复得》'}, {'作品': '《外星人E.T.》'}, {'作品': '《费城故事》'}]}, {'获奖经历': [{'奖项': '奥斯卡金像奖'}, {'奖项': '金球奖'}, {'奖项': 'BAFTA奖'}]}]}]}


# 4、列表解析器 CommaSeparatedListOutputParser

举例1：

In [23]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

messages = "大象,猩猩,狮子"
result = output_parser.parse(messages)
print(result)
print(type(result))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['大象', '猩猩', '狮子']
<class 'list'>


举例2：

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 创建解析器
output_parser = CommaSeparatedListOutputParser()

# 创建LangChain提示模板
chat_prompt = PromptTemplate.from_template(
    "生成5个关于{text}的列表.\n\n{format_instructions}",
    partial_variables={
    "format_instructions": output_parser.get_format_instructions()
    })

# 提示模板与输出解析器传递输出
# chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())

# 将提示和模型合并以进行调用
chain = chat_prompt | chat_model | output_parser
res = chain.invoke({"text": "电影"})
print(res)
print(type(res))

['1. 动作片', '科幻片', '恐怖片', '喜剧片', '爱情片  ', '2. 经典电影', '现代电影', '外国电影', '动画电影', '纪录片  ', '3. 导演', '编剧', '主演', '配角', '制片人  ', '4. 票房收入', '影评评分', '观众评分', '参赛奖项', '影展获奖  ', '5. 电影类型', '放映时间', '电影时长', '电影评分', '观看方式  ']
<class 'list'>


# 5、日期解析器 DatetimeOutputParser

举例1：

In [25]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1853-04-07T06:45:24.945854Z, 0835-04-16T12:35:51.978680Z, 1877-10-31T12:04:45.397316Z

Return ONLY this string, no other words!


举例2：


In [26]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser

chat_model = ChatOpenAI(model="gpt-4o-mini")


chat_prompt = ChatPromptTemplate.from_messages([
    ("system","{format_instructions}"),
    ("human", "{request}")
])

output_parser = DatetimeOutputParser()

# 方式1：
# model_request = chat_prompt.format_messages(
#     request="中华人民共和国是什么时候成立的",
#     format_instructions=output_parser.get_format_instructions()
# )

# response = chat_model.invoke(model_request)
# result = output_parser.invoke(response)
# print(result)
# print(type(result))

# 方式2：
chain = chat_prompt | chat_model | output_parser
resp = chain.invoke({"request":"中华人民共和国是什么时候成立的",
                     "format_instructions":output_parser.get_format_instructions()})
print(resp)
print(type(resp))

1949-10-01 00:00:00
<class 'datetime.datetime'>
